1. Import library

In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

2. Read data from CSV and train model

In [15]:
cryptos = ['ADA', 'BTC', 'ETH']

for crypto in cryptos:
    data = pd.read_csv(f'../data/processed/{crypto}_processed.csv')

    data = data[['Date', 'Close']]

    data['Date'] = pd.to_datetime(data['Date'])

    data.set_index('Date', inplace=True)

    scaler = MinMaxScaler(feature_range=(0, 1))
    final_data = data.values
    scaled_data = scaler.fit_transform(final_data)

    train_size = int(len(scaled_data) * 0.7)
    train_data = scaled_data[:train_size]
    valid_data = scaled_data[train_size:]

    train_df = pd.DataFrame(train_data, columns=['Close scaled'])
    valid_df = pd.DataFrame(valid_data, columns=['Close scaled'])
    train_df.to_csv(f'../data/split/train/{crypto}_train.csv', index=False)
    valid_df.to_csv(f'../data/split/validation/{crypto}_validation.csv', index=False)

    x_train, y_train = [], []
    for i in range(60, len(train_data)):
        x_train.append(train_data[i-60:i, 0])
        y_train.append(train_data[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dense(units=1))

    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(x_train, y_train, epochs=1, batch_size=1)

    model.save(f'../models/saved_models/{crypto}_model.h5')


c:\Users\ngqgh\OneDrive\Desktop\stock-price-prediction\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


707/707 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - loss: 0.0093


c:\Users\ngqgh\OneDrive\Desktop\stock-price-prediction\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


707/707 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - loss: 0.0132


c:\Users\ngqgh\OneDrive\Desktop\stock-price-prediction\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


706/706 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 0.0147
